In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from data_loader import download_stock
from feature_engineering import add_features
from models_ml import train_linear_regression, train_random_forest, evaluate
from model_lstm import prepare_lstm_data, create_lstm_model
from utils import save_plot

# Step 1: Download
df = download_stock("AAPL")
df = add_features(df)

# Step 2: ML Models
features = ["Open", "High", "Low", "Close", "Volume", "MA10", "MA50", "EMA20", "RSI"]
X = df[features]
y = df["Close"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Linear Regression
lr = train_linear_regression(X_train, y_train)
lr_rmse, lr_pred = evaluate(lr, X_test, y_test)

# Random Forest
rf = train_random_forest(X_train, y_train)
rf_rmse, rf_pred = evaluate(rf, X_test, y_test)

save_plot(y_test.values, lr_pred, "lr_plot.png")
save_plot(y_test.values, rf_pred, "rf_plot.png")

# Step 3: LSTM
scaler = MinMaxScaler()
scaled = scaler.fit_transform(df["Close"].values.reshape(-1,1))

X_lstm, y_lstm = prepare_lstm_data(scaled)

split = int(0.8 * len(X_lstm))
X_train_lstm, X_test_lstm = X_lstm[:split], X_lstm[split:]
y_train_lstm, y_test_lstm = y_lstm[:split], y_lstm[split:]

model = create_lstm_model()
model.fit(X_train_lstm, y_train_lstm, epochs=10, batch_size=32, verbose=1)

pred_lstm = model.predict(X_test_lstm)
save_plot(y_test_lstm, pred_lstm, "lstm_plot.png")

print("Training Complete!")


ModuleNotFoundError: No module named 'data_loader'